# Satellite Orbit Calculation for Observation Planning
Written by Kiyoaki Okudaira<br>
*Kyushu University Hanada Lab / University of Washington / IAU CPS SatHub<br>
(okudaira.kiyoaki.528@s.kyushu-u.ac.jp or kiyoaki@uw.edu)<br>
<br>
Calculate satellite orbit by SGP4 propagation for ground-based observation and plot az-el & pha-el<br>
<br>
**History**<br>
coding 2025-12-07 : 1st coding<br>
<br>
(c) 2025 Kiyoaki Okudaira - Kyushu University Hanada Lab (SSDL) / University of Washington / IAU CPS SatHub

### Parameters
**Observation date**

In [ ]:
obs_begin = "2025-01-21T09:00:00" # UTC Date and time observation begin [YYYY-MM-DDTHH:MM:SS.SSS] | str
obs_end   = "2025-01-21T11:00:00" # UTC Date and time observation end   [YYYY-MM-DDTHH:MM:SS.SSS] | str
obs_step  = 240.0                 # Time step [sec] | float or int

**TLE Download settings**<br>
Downloading from space-track.org is highly recommended

In [ ]:
from input.satlist.EKRAN_2 import * # Satellite list

tle_source  = "space-track.org" # "space-track.org" or "celestrak.org" | str

### Import and initial settings

**PATH setting**

In [ ]:
base_PATH = "/Users/kiyoaki/VScode/satphotometry_package/"
output_PATH = base_PATH + "output"
input_PATH = base_PATH + "input"
spice_myfile_PATH = "/Users/kiyoaki/VScode/satphotometry_package/config/myfile.txt"
tle_PATH = "/Users/kiyoaki/VScode/satphotometry_package/input/tle/13056.txt" # If download_tle is False | str

**Standard libraries**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
from astropy.table import Table

**Satphotometry library**<br>
satphotometry.satorbit must be imported after starting up SPICE kernel

In [ ]:
from satphotometry import gettle

**Observatory setting**

In [ ]:
from input.obs_site.KUPT import *
# obs_gd_lon_deg = 130.0 + (12.0 + 42.0 / 60.0) / 60.0    # [deg]
# obs_gd_lat_deg = 33.0 + (35.0 + 56.0 / 60.0) / 60.0     # [deg]
# obs_gd_height  = 0.073  # [km]

# wavelength_m = 0.5e-6   #[m]
# aperture_m   = 508.0e-3 #[m]

obs_params = [obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,wavelength_m,aperture_m]

**Space-track.org settings**<br>
Set your space-track.org identity and password at `./config/space-track-org.config`

In [ ]:
if tle_source == "space-track.org":
    with open(f'{base_PATH}config/space-track-org.config') as f:
        st_config = json.load(f)
        st_user_id  = st_config["identity"] # space-track.org user id | str
        st_password = st_config["password"] # space-track.org password | str

### Orbit calculation

**Output list**

In [ ]:
outputs = []
norad_ids_new = []
intldess = []
objnames = []

**Download TLE file**<br>
Download latest Two-Line Element set from space-track.org or celestrak.org

In [ ]:
tle_PATHs = []
for norad_id in norad_ids:
    status = 0
    if tle_source == "space-track.org":
        status,tle_result = gettle.space_track.get_latest_TLE(norad_id,st_user_id,st_password)
    if tle_source != "space-track.org" or status != 200:
        status,tle_result = gettle.celes_trak.get_latest_TLE(norad_id)

    if status == 200:
        tle_fname = "{0}_{1}".format(norad_id,gettle.parse_tle2epoch_fname(tle_result.splitlines()[1]))
        tle_PATH = "{0}/tle/{1}.tle".format(input_PATH,tle_fname)
        with open(tle_PATH,mode="w") as f:
            f.write(tle_result)
        tle_PATHs.append(tle_PATH)
    else:
        raise ValueError("Failed to download Two-Line Element set or NORAD catalog number is invalid.")

**SGP4 Propagation**

In [ ]:
import cal_satorbit
for tle_PATH in tle_PATHs:
    objname,norad_id,intldes,output = cal_satorbit.cal_satorbit(obs_begin,obs_end,obs_step,tle_PATH,obs_params,spice_myfile_PATH)
    outputs.append(output)
    norad_ids_new.append(norad_id)
    intldess.append(intldes)
    objnames.append(objname)

### Output
**Plot**

In [ ]:
# az-el
plt.figure(figsize=(12,6))
for i in range(0,len(outputs)):
    output = Table(outputs[i])
    norad_id = norad_ids_new[i]
    intldes = intldess[i]
    objname = objnames[i]

    output_filtered = output[(output["umbra"] == False)]

    plt.plot(output_filtered["az[deg]"], output_filtered["el[deg]"], color='black')
    plt.scatter(output_filtered["az[deg]"][0], output_filtered["el[deg]"][0],marker='.',color='blue')
    plt.scatter(output_filtered["az[deg]"][-1], output_filtered["el[deg]"][-1],marker='.',color='red')
    plt.annotate(" {0}".format(intldes if legend_view == "INTLDES" else objname),(output_filtered["az[deg]"][0], output_filtered["el[deg]"][0]),fontsize=4)
    plt.annotate('', xy=(output_filtered["az[deg]"][-1], output_filtered["el[deg]"][-1]), xytext=(output_filtered["az[deg]"][-2], output_filtered["el[deg]"][-2]),
                arrowprops=dict(shrink=0, width=1, headwidth=2, 
                                headlength=3, connectionstyle='arc3',
                                facecolor='black', edgecolor='black')
                )
plt.scatter(-99,-99,marker='.',color='blue',label=output_filtered["YYYY-MM-DDThh:mm:ss"][0])
plt.scatter(-99,-99,marker='.',color='red',label=output_filtered["YYYY-MM-DDThh:mm:ss"][-1])
plt.xlabel('Azimuth [deg]')
plt.ylabel('Elevation [deg]')
plt.xlim(az_min,az_max)
plt.ylim(0,90)
plt.xticks(np.arange(az_min,az_max+az_interval,az_interval))
plt.yticks(np.arange(0,100,10))
plt.grid()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=2)
plt.savefig("{0}/orbit/EKRAN_{1}_az-al.png".format(output_PATH,obs_begin[0:10]),dpi=640)
plt.savefig("{0}/orbit/EKRAN_{1}_az-al.pdf".format(output_PATH,obs_begin[0:10]))
plt.show()

# pha-el
plt.figure(figsize=(12,6))
for i in range(0,len(outputs)):
    output = Table(outputs[i])
    norad_id = norad_ids_new[i]
    intldes = intldess[i]
    objname = objnames[i]

    output_filtered = output[(output["umbra"] == False)]

    plt.plot(output_filtered["pha[deg]"], output_filtered["el[deg]"], color='black')
    plt.scatter(output_filtered["pha[deg]"][0], output_filtered["el[deg]"][0],marker='.',color='blue')
    plt.scatter(output_filtered["pha[deg]"][-1], output_filtered["el[deg]"][-1],marker='.',color='red')
    plt.annotate(" {0}".format(intldes if legend_view == "INTLDES" else objname),(output_filtered["pha[deg]"][0], output_filtered["el[deg]"][0]),fontsize=4)
    plt.annotate('', xy=(output_filtered["pha[deg]"][-1], output_filtered["el[deg]"][-1]), xytext=(output_filtered["pha[deg]"][-2], output_filtered["el[deg]"][-2]),
                arrowprops=dict(shrink=0, width=1, headwidth=2, 
                                headlength=3, connectionstyle='arc3',
                                facecolor='black', edgecolor='black')
                )
plt.scatter(-99,-99,marker='.',color='blue',label=output_filtered["YYYY-MM-DDThh:mm:ss"][0])
plt.scatter(-99,-99,marker='.',color='red',label=output_filtered["YYYY-MM-DDThh:mm:ss"][-1])
plt.xlabel('Solar Phase Angle [deg]')
plt.ylabel('Elevation [deg]')
plt.xlim(0,180)
plt.ylim(0,90)
plt.xticks(np.arange(0,210,30))
plt.yticks(np.arange(0,100,10))
plt.grid()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=2)
plt.savefig("{0}/orbit/{1}_{2}_pha-al.png".format(output_PATH,ftitle,obs_begin[0:10]),dpi=640)
plt.savefig("{0}/orbit/{1}_{2}_pha-al.pdf".format(output_PATH,ftitle,obs_begin[0:10]))
plt.show()